In [14]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import col


StatementMeta(, 970f2fec-9d25-4279-8b81-8db923960a18, 16, Finished, Available)

Now we will be creating a Order Priority Table

In [16]:
DeltaTable.createIfNotExists(spark)\
           .tableName('gold_order_priority')\
           .addColumn('Order_Priority_ID', LongType())\
           .addColumn('Order_Priority', StringType())\
           .addColumn('Create_TS', TimestampType())\
           .addColumn('Modified_TS', TimestampType())\
           .execute()

StatementMeta(, 970f2fec-9d25-4279-8b81-8db923960a18, 18, Finished, Available)

In [17]:
# incremental logic table
df = spark.read.table('project_lakehouse.gold_order_priority')

StatementMeta(, 970f2fec-9d25-4279-8b81-8db923960a18, 19, Finished, Available)

In [18]:
Max_date = df.selectExpr("coalesce(max('Modified_TS'), '1900-01-01')").first()[0]
Max_date

StatementMeta(, 970f2fec-9d25-4279-8b81-8db923960a18, 20, Finished, Available)

'1900-01-01'

In [19]:
df_bronze = spark.read.table('project_lakehouse.bronze_sales')


StatementMeta(, 970f2fec-9d25-4279-8b81-8db923960a18, 21, Finished, Available)

In [22]:
df_bronze.select('Order_Priority').distinct().show()


StatementMeta(, 970f2fec-9d25-4279-8b81-8db923960a18, 24, Finished, Available)

+--------------+
|Order_Priority|
+--------------+
|          High|
|        Medium|
|      Critical|
|           Low|
+--------------+



In [23]:
df_bronze_mod = df_bronze.select('Order_Priority').where(col("Modified_TS")>Max_date).distinct()


StatementMeta(, 970f2fec-9d25-4279-8b81-8db923960a18, 25, Finished, Available)

In [24]:
Max_ID = df.selectExpr("coalesce(max('Order_Priority_ID'), 0)").first()[0]
Max_ID

StatementMeta(, 970f2fec-9d25-4279-8b81-8db923960a18, 26, Finished, Available)

'0'

In [25]:
# now set incremental column id using monotonically incresing ID fucntion
df_final = df_bronze_mod.withColumn("Order_Priority_ID", Max_ID+monotonically_increasing_id()+1)

StatementMeta(, 970f2fec-9d25-4279-8b81-8db923960a18, 27, Finished, Available)

In [26]:
df_final.show()

StatementMeta(, 970f2fec-9d25-4279-8b81-8db923960a18, 28, Finished, Available)

+--------------+-----------------+
|Order_Priority|Order_Priority_ID|
+--------------+-----------------+
|      Critical|              1.0|
|          High|              2.0|
|        Medium|              3.0|
|           Low|              4.0|
+--------------+-----------------+



In [42]:
df_gold_delta = DeltaTable.forPath(spark, "Tables/gold_order_priority")
df_bronze_table = df_final

StatementMeta(, 970f2fec-9d25-4279-8b81-8db923960a18, 44, Finished, Available)

In [50]:
df_gold_delta.alias("gold")\
             .merge(\
               df_bronze_table.alias("Bronze"),\
                "gold.Order_Priority==Bronze.Order_Priority"\
             )\
             .whenMatchedUpdate(\
                  set={
                    "gold.Modified_TS":current_timestamp()
                  }
             )\
             .whenNotMatchedInsert(\
                    values={
                         "gold.Order_Priority_ID":"Bronze.Order_Priority_ID",
                     "gold.Order_Priority":"Bronze.Order_Priority",
                     "gold.Create_TS":current_timestamp(),
                     "gold.Modified_TS":current_timestamp()
                    }
             )\
             .execute()

StatementMeta(, 970f2fec-9d25-4279-8b81-8db923960a18, 52, Finished, Available)